In [9]:
import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

# load in directories
from experiment_code.constants import Defaults
from experiment_code import preprocess
from experiment_code.targetfile_utils import Utils
from experiment_code.preprocess import ExpSentences
from experiment_code.visualization.visualize import CoRTLanguageExp

In [10]:
cort = CoRTLanguageExp() 
df = cort.load_dataframe()
df

,local_date,experiment_id,experiment_version,participant_public_id,participant_id,task_name,task_version,spreadsheet_version,spreadsheet_row,sentence_num,...,negative,tense,spelling_modified,trial_type,version,version_descript,group_cloze_condition,group_CoRT_condition,group_trial_type,cloze_cort
311,20/07/2020 16:18:19,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,222.0,25,...,no,past,no,meaningless,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: high cloze,control: non-CoRT,control: meaningless,"non-CoRT, high cloze"
324,20/07/2020 16:18:26,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,223.0,26,...,no,past,no,meaningless,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: high cloze,control: CoRT,control: meaningless,"CoRT, high cloze"
337,20/07/2020 16:18:32,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,224.0,27,...,no,past,no,meaningful,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: low cloze,control: non-CoRT,control: meaningful,"non-CoRT, low cloze"
350,20/07/2020 16:18:39,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,225.0,28,...,no,past,yes,meaningful,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: low cloze,control: non-CoRT,control: meaningful,"non-CoRT, low cloze"
363,20/07/2020 16:18:45,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,226.0,29,...,no,past,no,meaningful,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: low cloze,control: CoRT,control: meaningful,"CoRT, low cloze"
374,20/07/2020 16:18:51,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,227.0,30,...,no,past,no,meaningless,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: low cloze,control: non-CoRT,control: meaningless,"non-CoRT, low cloze"
385,20/07/2020 16:18:56,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,228.0,31,...,no,past,no,meaningful,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: high cloze,control: non-CoRT,control: meaningful,"non-CoRT, high cloze"
397,20/07/2020 16:19:02,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,229.0,32,...,no,past,no,meaningless,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: low cloze,control: CoRT,control: meaningless,"CoRT, low cloze"
409,20/07/2020 16:19:08,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,230.0,33,...,no,past,no,meaningless,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: low cloze,control: non-CoRT,control: meaningless,"non-CoRT, low cloze"
421,20/07/2020 16:19:14,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,231.0,34,...,no,present,yes,meaningless,10,CONCAT OF 7-9. Shortened to 5 runs (experiment),control: high cloze,control: non-CoRT,control: meaningless,"non-CoRT, high cloze"


In [13]:
out_dir = os.path.join(Defaults.PROCESSED_DIR, "preprocessed_dataframe")
df.to_csv(out_dir) 

## Reviewed df stats

In [25]:
df = pd.read_csv(os.path.join(Defaults.RAW_DIR, f"cort_language_gorilla_v7.csv"))

In [26]:
def _get_response_type():
    response_type = "response_keyboard_single"
    return response_type

def _assign_trialtype(x):
    if x==False:
        value = "meaningful"
    elif x==True:
        value = "meaningless"
    else:
        value = x
    return value

def _rename_cols(df):
    """rename some columns for analysis
    """
    return df.rename({'Local Date':'local_date','Experiment ID':'experiment_id', 'Experiment Version':'experiment_version', 'Participant Public ID':'participant_public_id', 'Participant Private ID':'participant_id', 
                'Task Name':'task_name', 'Task Version':'task_version', 'Spreadsheet Name':'spreadsheet_version', 'Spreadsheet Row': 'spreadsheet_row', 'Trial Number': 'sentence_num', 'Zone Type':'zone_type', 
                'Reaction Time':'rt', 'Response':'response', 'Attempt':'attempt', 'Correct':'correct', 'Incorrect':'incorrect'}, axis=1)
    

# determine which cols to keep depending on task

df = df.filter({'Local Date', 'Experiment ID', 'Experiment Version', 'Participant Public ID', 'Participant Private ID',
        'Task Name', 'Task Version', 'Spreadsheet Name', 'Spreadsheet Row', 'Trial Number', 'Zone Type', 
        'Reaction Time', 'Response', 'Attempt', 'Correct', 'Incorrect', 'display', 'block_num', 'randomise_blocks',
        'full_sentence', 'last_word', 'sampled','CoRT_descript', 'CoRT_mean','condition_name',
        'CoRT_std','cloze_descript', 'cloze_probability', 'dataset', 'random_word', 'target_word', 'word_count'})

#rename some columns for analysis
df = _rename_cols(df)

# filter dataset to include trials and experimental blocks (i.e. not instructions)
response_type = _get_response_type() # response_type is different across the task
df = df.query(f'display=="trial" and block_num>0 and zone_type=="{response_type}"')
df['rt'] = df['rt'].astype(float)  

In [35]:
blocks = df.groupby('participant_id').apply(lambda x: x.sort_values('block_num').block_num).values
blocks

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 

In [30]:
df['block_num'] = blocks

ValueError: Length of values does not match length of index

In [ ]:
# correct block_num to be sequential
df['block_num'] = df.apply(lambda x: x.sort_values('block_num').block_num).values
        
# get meaningful assesment
df['trial_type'] = df['sampled'].apply(lambda x: _assign_trialtype(x))

# get version
df["version"] = version

# get condition name (make sure it's just characters)
df['condition_name'] = df['condition_name'].str.extract('([a-zA-Z]*)')

df